In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [2]:
data=fetch_california_housing()
x=data.data
y=data.target


In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

In [4]:
xtrain,xtest,ytrain,ytest=train_test_split(X_scaled,y,test_size=0.2)

In [5]:
xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')
ytrain = ytrain.astype('float32')
ytest = ytest.astype('float32')


In [6]:
xtrain = torch.FloatTensor(xtrain)
xtest = torch.FloatTensor(xtest)
ytrain = torch.FloatTensor(ytrain)
ytest = torch.FloatTensor(ytest)

In [ ]:
class Model(nn.Module):
    def __init__(self,input_features=8,hidden_layer1=25,hidden_layer2=30,output_features=1):
        super().__init__()
        self.fc1=nn.Linear(input_features,hidden_layer1)
        self.fc2=nn.Linear(hidden_layer1,hidden_layer2)
        self.out=nn.Linear(hidden_layer2,output_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

model=Model()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 100
losses = []

for i in range(epochs):

    y_pred = model.forward(xtrain)
    loss= criterion(y_pred, ytrain)
    losses.append(loss.detach())
    print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('epoch')

preds = []
with torch.no_grad():
    for val in xtest:
        y_hat = model.forward(val)
        preds.append(y_hat.argmax().item())


In [ ]:
inputs_new = torch.tensor(xtest, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    predictions = model(inputs_new)

# Convert predictions tensor to numpy array
predictions = predictions.numpy()

# Print the predictions
print(predictions)

In [9]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(ytest, predictions)
print("Mean Squared Error:", mse)


Mean Squared Error: 1.3161376
